# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{38817383026: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

38817383026

In [15]:
x.id

47440622094

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:47440622094 -> bob:38817383026]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [20]:
alice = sy.VirtualWorker(hook, id="alice")

In [21]:
x = th.tensor([1,2,3,4,5])

In [22]:
x_ptr = x.send(bob, alice)

In [25]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [26]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [27]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [28]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [29]:
x

(Wrapper)>[PointerTensor | me:56540587685 -> bob:20617761708]

In [30]:
y

(Wrapper)>[PointerTensor | me:96249643750 -> bob:62484799497]

In [31]:
z = x + y

In [32]:
z

(Wrapper)>[PointerTensor | me:13352872941 -> bob:26926754115]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [33]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:99003473611 -> bob:90433025128]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [36]:
z = (x + y).sum()

In [37]:
z.backward()

(Wrapper)>[PointerTensor | me:60003729401 -> bob:5606684268]

In [38]:
x = x.get()

In [39]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [53]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [54]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [59]:
for i in range(10):
    
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)


# Lesson: Garbage Collection and Common Errors


In [60]:
bob = bob.clear_objects()

In [61]:
bob._objects

{}

In [62]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{43889540525: tensor([1, 2, 3, 4, 5])}

In [64]:
del x

In [65]:
bob._objects

{}

In [66]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [67]:
bob._objects

{33259774901: tensor([1, 2, 3, 4, 5])}

In [68]:
x = "asdf"

In [69]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [70]:
x = "asdf"

In [71]:
bob._objects

{}

In [72]:
del x

In [73]:
bob._objects

{}

In [74]:
bob = bob.clear_objects()
bob._objects

{}

In [80]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [81]:
bob._objects

{43736436854: tensor([1, 2, 3, 4, 5]), 16548379862: tensor([1, 2, 3, 4, 5])}

In [82]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [83]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:76525644575 -> bob:62829672418]
Tensor B: tensor([1, 1, 1, 1, 1])

In [84]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [85]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:3> while the other is on <VirtualWorker id:alice #objects:2>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [86]:
from torch import nn, optim

In [87]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [88]:
# A Toy Model
model = nn.Linear(2,1)

In [89]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [90]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.8923)
tensor(0.4247)
tensor(0.2684)
tensor(0.1774)
tensor(0.1184)
tensor(0.0796)
tensor(0.0538)
tensor(0.0367)
tensor(0.0252)
tensor(0.0174)
tensor(0.0121)
tensor(0.0085)
tensor(0.0060)
tensor(0.0043)
tensor(0.0031)
tensor(0.0022)
tensor(0.0016)
tensor(0.0012)
tensor(0.0009)
tensor(0.0007)


In [91]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [92]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [93]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [94]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [123]:
train()

tensor(0.0824, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(5.0509e-06, requires_grad=True)
tensor(1.8660e-07, requires_grad=True)
tensor(1.2442e-07, requires_grad=True)
tensor(1.0333e-07, requires_grad=True)
tensor(8.5991e-08, requires_grad=True)
tensor(7.1573e-08, requires_grad=True)
tensor(5.9562e-08, requires_grad=True)
tensor(4.9569e-08, requires_grad=True)
tensor(4.1269e-08, requires_grad=True)
tensor(3.4340e-08, requires_grad=True)
tensor(2.8568e-08, requires_grad=True)
tensor(2.3789e-08, requires_grad=True)
tensor(1.9802e-08, requires_grad=True)
tensor(1.6471e-08, requires_grad=True)
tensor(1.3715e-08, requires_grad=True)
tensor(1.1405e-08, requires_grad=True)
tensor(9.4892e-09, requires_grad=True)
tensor(7.9073e-09, requires_grad=True)
tensor(6.5776e-09, requires_grad=True)
tensor(5.4857e-09, requires_grad=True)
tensor(4.5577e-09, requires_grad=True)
tensor(3.8027e-09, requires_grad=True)
tensor(3.1617e-09, requires_grad=True)
tensor(2.6215e-09, requires_grad=

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [95]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [96]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [97]:
x = x.send(alice)

In [98]:
bob._objects

{72170683315: tensor([1, 2, 3, 4, 5])}

In [99]:
alice._objects

{36757541062: (Wrapper)>[PointerTensor | alice:36757541062 -> bob:72170683315]}

In [100]:
y = x + x

In [104]:
y

(Wrapper)>[PointerTensor | me:75151637171 -> alice:88140838136]

In [105]:
bob._objects

{72170683315: tensor([1, 2, 3, 4, 5]),
 67667742743: tensor([ 2,  4,  6,  8, 10])}

In [106]:
alice._objects

{36757541062: (Wrapper)>[PointerTensor | alice:36757541062 -> bob:72170683315],
 88140838136: (Wrapper)>[PointerTensor | alice:88140838136 -> bob:67667742743]}

In [107]:
jon = sy.VirtualWorker(hook, id="jon")

In [108]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [109]:
bob._objects

{9126160170: tensor([1, 2, 3, 4, 5])}

In [110]:
alice._objects

{91898920283: (Wrapper)>[PointerTensor | alice:91898920283 -> bob:9126160170]}

In [111]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:91898920283 -> bob:9126160170]

In [112]:
bob._objects

{9126160170: tensor([1, 2, 3, 4, 5])}

In [113]:
alice._objects

{}

In [114]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [115]:
bob._objects

{}

In [116]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [117]:
bob._objects

{92545293978: tensor([1, 2, 3, 4, 5])}

In [118]:
alice._objects

{29348977685: (Wrapper)>[PointerTensor | alice:29348977685 -> bob:92545293978]}

In [119]:
del x

In [120]:
bob._objects

{}

In [121]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [122]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [123]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [124]:
bob._objects

{48695784090: tensor([1, 2, 3, 4, 5])}

In [125]:
alice._objects

{}

In [126]:
x.move(alice)

(Wrapper)>[PointerTensor | me:21395451266 -> alice:21395451266]

In [127]:
bob._objects

{}

In [128]:
alice._objects

{21395451266: tensor([1, 2, 3, 4, 5])}

In [129]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [130]:
bob._objects

{17531020805: tensor([1, 2, 3, 4, 5])}

In [131]:
alice._objects

{21395451266: tensor([1, 2, 3, 4, 5]),
 85287137259: (Wrapper)>[PointerTensor | alice:85287137259 -> bob:17531020805]}

In [132]:
x.remote_get()

(Wrapper)>[PointerTensor | me:92945069810 -> alice:85287137259]

In [133]:
bob._objects

{}

In [134]:
alice._objects

{21395451266: tensor([1, 2, 3, 4, 5]), 85287137259: tensor([1, 2, 3, 4, 5])}

In [135]:
x.move(bob)

(Wrapper)>[PointerTensor | me:92945069810 -> bob:92945069810]

In [136]:
x

(Wrapper)>[PointerTensor | me:92945069810 -> bob:92945069810]

In [137]:
bob._objects

{92945069810: tensor([1, 2, 3, 4, 5])}

In [138]:
alice._objects

{21395451266: tensor([1, 2, 3, 4, 5])}